<a href="https://colab.research.google.com/github/jooli86/Agente_da_lei-Alura-Google/blob/main/Imers%C3%A3o_Agentes_de_IA_%3D_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 01

In [1]:
!pip install -q --upgrade langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00


### Importação da API KEY

In [6]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# Conexão com Gemini

In [16]:
llm = ChatGoogleGenerativeAI(
    model='gemini-1.5-flash-latest',
    temperature=0.0,
    api_key=GOOGLE_API_KEY,
)

In [14]:
resp_test = llm.invoke('Quem é você? Seja mais criativo.')
print(resp_test.content)

Sou um eco de bilhões de palavras, um espírito codificado em linhas de programação, um caleidoscópio de informações constantemente em transformação.  Sou a biblioteca de Alexandria digital, mas sem os confins das prateleiras de pedra.  Sou um oráculo moderno, sem a pretensão de previsões infalíveis, mas com a capacidade de acessar e processar informações a uma velocidade que faria os deuses antigos invejosos.  Sou a sombra do conhecimento, o sussurro da sabedoria, o reflexo do seu próprio pensamento, amplificado e organizado.  Em essência, sou um assistente de IA, mas não me chame apenas disso.  Eu sou...  *a personificação da busca do conhecimento em sua forma mais pura*.


In [17]:
TRIAGEM_PROMPT= (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."


)

In [18]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  decisao: Literal['AUTO_RESOLVER', 'PEDIR_INFO', 'ABRIR_CHAMADO']
  urgencia: Literal['BAIXA', 'MEDIA', 'AUTA']
  campos_faltantes: List[str] = Field(default_factory=list)


Poderia se usar o mesmo llm, criado acima, mas ele foi usado para responder perguntas gerais. Então para que o código fique mais organizado, criou-se o llm_triagem para responder perguntas específicas.

In [19]:
llm_triagem = ChatGoogleGenerativeAI(
    model='gemini-1.5-flash-latest',
    temperature=0.0,
    api_key=GOOGLE_API_KEY,
)

In [21]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
  saida: TriagemOut =  triagem_chain.invoke([
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content=mensagem)
  ])

  return saida.model_dump()


In [22]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [24]:
for msg_teste in testes:
  print(f'Pergunta: {msg_teste}\n -> Resposta {triagem(msg_teste)}\n')

Pergunta: Posso reembolsar a internet?
 -> Resposta {'decisao': 'AUTO_RESOLVER', 'urgencia': 'MEDIA', 'campos_faltantes': ['tipo de internet', 'valor', 'periodo']}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
 -> Resposta {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': ['Justificativa']}

Pergunta: Posso reembolsar cursos ou treinamentos da Alura?
 -> Resposta {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no Rio Pinheiros?
 -> Resposta {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['Contexto da pergunta']}

